# 3.2 Human GEX Data

In [1]:
from clustergrammer2 import net
df = {}

import numpy as np
import pandas as pd

clustergrammer2 backend version 0.2.7


In [2]:
from copy import deepcopy

In [3]:
filename = '../data/CITE-seq_data/GSE100866_CBMC_8K_13AB_10X-RNA_umi_HUMAN.csv.gz'
df['gex-ini'] = pd.read_csv(filename, compression='gzip', index_col=0)
df['gex-ini'].shape

(20400, 7339)

In [4]:
net.load_file('../data/CITE-seq_data/adt_ashz_trim_cats.txt')
df['adt-ini'] = net.export_df()
df['adt-ini'].shape

(10, 7265)

In [5]:
cols = df['adt-ini'].columns.tolist()
keep_cells = [x[0] for x in cols]
print(len(keep_cells))

7265


#### Remove HUMAN prefix from genes

In [6]:
rows = df['gex-ini'].index.tolist()
new_rows = [x.replace('HUMAN_','') for x in rows]
df['gex-ini'].index = new_rows

### Filter for trimmed cells only and arcsinh transform

In [7]:
df['gex-trim'] = df['gex-ini'][keep_cells]
df['gex-trim'] = np.arcsinh(df['gex-trim']/5)

#### Drop ribosomal and mitochondrial genes

In [8]:
print(df['gex-trim'].shape)
df['gex'] = deepcopy(df['gex-trim'])
all_genes = df['gex'].index.tolist()
print(len(all_genes))
keep_genes = [x for x in all_genes if 'RPL' not in x]
keep_genes = [x for x in keep_genes if 'RPS' not in x]
print(len(keep_genes))

df['gex'] = df['gex'].loc[keep_genes]
df['gex'].shape

# Removing Mitochondrial Genes
list_mito_genes = ['MTRNR2L11', 'MTRF1', 'MTRNR2L12', 'MTRNR2L13', 'MTRF1L', 'MTRNR2L6', 'MTRNR2L7',
                'MTRNR2L10', 'MTRNR2L8', 'MTRNR2L5', 'MTRNR2L1', 'MTRNR2L3', 'MTRNR2L4']

all_genes = df['gex'].index.tolist()
mito_genes = [x for x in all_genes if 'MT-' == x[:3] or 
             x.split('_')[0] in list_mito_genes]

print(mito_genes)

keep_genes = [x for x in all_genes if x not in mito_genes]
df['gex'] = df['gex'].loc[keep_genes]
print(df['gex'].shape)

# # normalize by UMI count
# barcode_umi_sum = df['ge'].sum()
# df['ge'] = df['ge'].div(barcode_umi_sum)

(20400, 7265)
20400
20223
['MT-ATP6', 'MT-ATP8', 'MT-CO1', 'MT-CO2', 'MT-CO3', 'MT-CYB', 'MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND4L', 'MT-ND5', 'MT-ND6', 'MT-RNR1', 'MT-RNR2', 'MT-TD', 'MT-TF', 'MT-TG', 'MT-TH', 'MT-TI', 'MT-TL1', 'MT-TP', 'MT-TT', 'MT-TW', 'MTRF1', 'MTRF1L', 'MTRNR2L1', 'MTRNR2L10', 'MTRNR2L11', 'MTRNR2L12', 'MTRNR2L3', 'MTRNR2L4', 'MTRNR2L5', 'MTRNR2L6', 'MTRNR2L7', 'MTRNR2L8']
(20187, 7265)


#### Transfer categories

In [9]:
cols = df['adt-ini'].columns.tolist()
ct_dict = {}
for inst_col in cols:
    ct_dict[inst_col[0]] = inst_col[1]

In [10]:
cols = df['gex'].columns.tolist()
new_cols = [(x, 'Cell Type: ' + ct_dict[x]) for x in cols]
df['gex-cat'] = deepcopy(df['gex'])
df['gex-cat'].columns = new_cols

#### Keep top 5K genes by sum

In [11]:
ser_sum = df['gex-cat'].sum(axis=1).sort_values(ascending=False)
keep_genes = ser_sum.index.tolist()[:5000]
df['gex-cat-filt'] = df['gex-cat'].loc[keep_genes]
df['gex-cat-filt'].shape

(5000, 7265)

In [12]:
net.load_df(df['gex-cat-filt'])
net.filter_N_top(inst_rc='row', N_top=100, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "S100A9", "ini": 100, "clust": 88, "rank": 37, "rankvar": 91, "…